## Excel Extraction through SpaCy

SpaCy uses trained neural network pipelines that can be used in regular laptops for different languages. The models are pretrained on various corpora from blogs, text, images and such. So, SpaCy is used to get the understanding of the data much more better than regex methodology. <br>

I could have used transformer based models like BeRT within Spacy for better accuracy. But in this use case with given examples, traditional CNN based model like en_core_web_sm was deemed enough with the results it created.  

In [1]:
import os
import pandas as pd
import spacy
import dateparser
from collections import defaultdict
import warnings
import openpyxl
warnings.filterwarnings("ignore")

In [2]:
def correct_item_codes_by_initials(dfd, product_col='product_name', item_col='item_code'):
    """
    Corrects mismatched item codes by matching product initials with item code prefixes.

    Args:
        df (pd.DataFrame): Input DataFrame with product and item code columns.
        product_col (str): Column name containing product names.
        item_col (str): Column name containing item codes.

    Returns:
        pd.DataFrame: DataFrame with corrected item codes.
    """

    #Extract initials and code prefixes
    dfd['Product_Initials'] = dfd[product_col].apply(lambda x: ''.join(word[0].upper() for word in str(x).split()))
    dfd['Code_Prefix'] = dfd[item_col].apply(lambda x: str(x).split('-')[0].upper())

    #Create possible combinations
    combos = dfd[[product_col, 'Product_Initials']].drop_duplicates().merge(
        dfd[[item_col, 'Code_Prefix']].drop_duplicates(), how='cross'
    )

    #Match where initials == code prefix
    matched = combos[combos['Product_Initials'] == combos['Code_Prefix']]
    match_map = matched.set_index(product_col)[item_col].to_dict()

    #Apply corrected item codes
    dfd['Corrected_Item_Code'] = dfd[product_col].map(match_map)

    #Fall back to original if no match found
    dfd['Corrected_Item_Code'] = dfd['Corrected_Item_Code'].fillna(dfd[item_col])

    return dfd.drop(['Product_Initials','Code_Prefix','item_code'],axis=1)#[[product_col, item_col, 'Corrected_Item_Code']]

In [16]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Folder with your email .txt files
EMAIL_DIR = "mail/"
records = []

def extract_info(text, vendor_name, po_number):
    doc = nlp(text)
    current_item = defaultdict(list)
    current_item["vendor"] = vendor_name
    current_item["po_number"] = po_number
    # Initialize units with an empty list to avoid the AttributeError
    current_item["units"] = []
    current_item["product_name"] = []
    current_item["item_code"] = []

    for sent in doc.sents:
        tokens = list(sent)
        # Simple way to catch product lines like "100 units of XYZ (Item Code: XYZ-123)"
        for i, token in enumerate(tokens):
            if token.like_num and i + 1 < len(tokens) and "unit" in tokens[i+1].text.lower():
                # Check if the previous token is also a number (extra/mistake)
                if i > 0 and tokens[i-1].like_num:
                    # Choose the first number (the one before)
                    chosen_num = int(tokens[i-1].text)
                else:
                    chosen_num = int(token.text)

                current_item["units"].append(chosen_num)

                # Get product name
                name_tokens = []
                j = i + 3  # skip "of"
                while j < len(tokens) and tokens[j].is_alpha:
                    name_tokens.append(tokens[j].text.title())
                    j += 1
                current_item["product_name"].append(" ".join(name_tokens))


        for i, token in enumerate(tokens):
            if token.text.lower() == "code":
                if i + 2 < len(tokens) and tokens[i+1].text in [":", "-", "is"]:
                    current_item["item_code"].append(tokens[i+2].text.strip(").,"))
                elif i + 1 < len(tokens):
                    current_item["item_code"].append(tokens[i+1].text.strip(").,"))

        # Store only complete lines with product name
        if current_item["product_name"]:
            records.append(current_item.copy())
            # Reset to empty lists for next item
            current_item["product_name"] = [] 
            current_item["units"] = []
            current_item["item_code"] = []

    for ent in doc.ents:
        if ent.label_ == "DATE":
            parsed_date = dateparser.parse(ent.text, settings={'PREFER_DATES_FROM': 'future'})
            if parsed_date:
                records[-1]['delivery_date'] = parsed_date.date().isoformat()


# Go through all .txt files
for file in os.listdir(EMAIL_DIR):
    if file.endswith(".txt"):
        filepath = os.path.join(EMAIL_DIR, file)
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()
            # Extract vendor and PO from filename
            name = os.path.splitext(file)[0]
            if "_PO_" in name:
                vendor, po = name.split("_PO_")
            else:
                vendor, po = name, ""
            extract_info(text, vendor.title(), po)

# Save to Excel
df = pd.DataFrame(records)
df = df.explode(['units', 'product_name', 'item_code'], ignore_index=False)
df = correct_item_codes_by_initials(df)
df.sort_values(by=['vendor','po_number'],ignore_index=True,inplace=True)
df['duplicate'] = df.duplicated(subset=['units', 'product_name', 'Corrected_Item_Code', 'delivery_date', 'vendor'], keep='first').astype(int)
df.to_excel("purchase_orders.xlsx", index=False)
print("Extraction complete. Data saved to 'purchase_orders.excel'.")

Extraction complete. Data saved to 'purchase_orders.excel'.
